# Title: Data Analysis Exercise: Our World in Data Junior Data Scientist Application
Author: Ruperto Pedro Bonet Chaple
Email: ruperto.bonet@gmail.com
Goal: Calculate the crude death rate and age-standardized rate for the United States and Uganda (1950-2021)


# Report about the Data Analysis Exercise: Our World in Data Junior Data Scientist Application

Tools:

a) Data Portal API using Python.
b) PDF files obtained from:
- Paper: Omar B. Ahmad, Cynthia Boscht-Pinto et al. "Age Standardization of Rates: A New WHO Standard". CPE Discussion Paper Series: No 31, EIP/GPE/EBD. World Health Organization 2001.
- https://owid.notion.site/Data-analysis-exercise-Our-World-in-Data-Junior-Data-Scientist-application-ab287a3c07264b4d91aadc436021b8c0

Methodology:

1) Use the WHO API to retireve data on the indicator 46: "Population estimates (variant: Median) of the United States and Uganda (1950-2021)
2) Data mining to obtain the Population Estimates Table (pop_est dataFrame).
3) Extract tables from the PDFs using camelot-py[all] Python tool to build the
dataframes age_st ("Age Standard Population Distribution (%)") and death_rate ("Age-Specific Death Rates of COPD Table").

Assumptions:

1) The aggregation of data for the Population Estimates over age 85 is cumulative.

2) The Age Standard Population belongs to the Total Population for the calculation of the Age-Specific Death Rate (ASDR).

Results:

1) A comparison between the Crude Death Rate (CDR) results and the results published by WHO in 2019 showed notable differences in the United States CDR. The aggregation of data for ages over 85 may be the primary cause of this discrepancy.

2) The age-standardized death rates showed similar results between both countries using different measures: Segi ("world") standard Scandinavian and ("European") standard WHO World Standard*. However, there are significant differences between them despite having similar means.

Difficulties:

1)  Several technical difficulties were encountered with the internet provider, and,

2) finding the right Camelot version available for Python 3.10 in Google Colab.

Observations:

The notebook Population_WHO_un_1.ipynb shows the deadline time according to the techical difficulties

# Introduction:
# Crude Death Rate (CDR)
The crude death rate (CDR) is a basic measure used in demography and public health to provide an estimate of the number of deaths in a population over a specific period of time, usually expressed per 1,000 individuals per year
# Age-standardized death rates(ASDRs)
 The age-standardized death rates (ASDRs) is a method used to compare mortality rates across populations with different age structures. It's particularly useful for comparing the mortality rates of different regions or over time within the same region, where the age distribution might change. The basic idea is to apply the observed age-specific death rates of a population to a standard population structure, so that differences observed in the standardized rates are not due to differences in the age distribution of the populations being compared.

In [ ]:

import pandas as pd
import requests
import json
import urllib3
import ssl


UPLOADING FILES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/DATA_TESTS/age_st.pdf /content/
!cp -r /content/drive/MyDrive/DATA_TESTS/dataexe01.pdf /content/

In [ ]:
!pip install pypdf2==3.0.1

In [ ]:
Path ='/content/age_st.pdf'
Path1='/content/dataexe01.pdf'

In [ ]:
!pip install -U camelot-py==0.11.0

In [ ]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

# Define a function that order a DataFrame by the column with range ['a-b']

In [ ]:
def data_range_sort(data:pd.DataFrame(), index_col,reverse:bool=False):
                  def tt(x):
                      for ch in ['-','+']:
                          if ch in x:
                              return int(x.replace(ch,' ').split()[0])
                  newd=[(ii,tt(y)) for ii,y in enumerate(data[data.columns[index_col]])]
                  newd.sort(key=lambda x:x[1],reverse=reverse)
                  new_order=[item[0] for item in newd]
                  return data.loc[new_order,:]

Define a function to extract a table from a PDF  document

In [ ]:
def extract_table_from_pdf(path,page_number:int,flavor:str="stream",stdout:bool=True):
           import camelot

           return camelot.read_pdf(path,flavor="stream",suppress_stdout=True,
                        pages="all")[page_number].df


# Define a function that will take a relative path as an input, call the API, and return a dataframe


In [ ]:
def callAPI(relative_path:str, topic_list:bool = False) -> pd.DataFrame:



    base_url = "https://population.un.org/dataportalapi/api/v1"

    target = base_url + relative_path # Query string parameters may be appended here or directly in the provided relative path
    # Calls the API
    #response = requests.get(target, verify=False)
    response = get_legacy_session().get(target)

    # Reformats response into a JSON object
    j = response.json()

    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
    # If results are paginated, they are transformed into a python dictionary.
    # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j['data'])
        # As long as the nextPage key of the dictionary contains an address for the next API call, the function will continue to call the API and append the results to the dataframe.
        while j['nextPage'] is not None:
                #response = requests.get(j['nextPage'])
                response = get_legacy_session().get(j['nextPage'])
                j = response.json()
                df_temp = pd.json_normalize(j['data'])
                df = df._append(df_temp)
    except:
           if topic_list:
                df = pd.json_normalize(j, 'indicators')
           else:
                df = pd.DataFrame(j)
    return(df)


# Uses callAPI function to get a list of parameters of Population Estimates(1950-2021) in the locations : United States and Uganda

In [ ]:
df_ily = callAPI("/data/indicators/46/locations/800,840/start/1950/end/2021")


# Extract he values of population with the variant "Median" acording the indicator Id=46 :Population by 5-year age groups and sex
# Get the values of population estimates of Uganda (1950-2021) based on the 5-year-age groups and sex


In [ ]:
UGA_pop=df_ily.loc[df_ily["locationId"]==800][["ageLabel","value"]].groupby("ageLabel").sum().reset_index()


 Get the values of population estimates of USA (1950-2021) based on the 5-year-age groups and sex

In [ ]:
USA_pop=df_ily.loc[df_ily["locationId"]==840][["ageLabel","value"]].groupby("ageLabel").sum().reset_index()

# Build a table concatenating these dataframes: USA_pop and UGA_pop

In [ ]:
pop_estimates=pd.concat([USA_pop,UGA_pop],axis=1)

 Drop duplicate columns "ageLabel"

In [ ]:
pop_estimates=pop_estimates.T.drop_duplicates().T

#Sort the DataFrame by the columns "ageLabel"

In [ ]:
pop_est=data_range_sort(pop_estimates,0)

Rename the columns of pop_estimates with the columns:["Age group(years)","United States,1950-2021","Uganda,1950-2021"])

In [ ]:
pop_est.columns=["Age group(years)","United States,1950-2021","Uganda,1950-2021"]
pop_est=pop_est.reset_index(drop=True)
print(pop_est)
pop_est.to_csv("pop_est.csv")


   Age group(years) United States,1950-2021 Uganda,1950-2021
0               0-4            2665210640.0      519693594.0
1               5-9            2668027455.0      430624776.0
2             10-14            2679617305.0      364621694.0
3             15-19            2641486119.0      304857173.0
4             20-24            2577774727.0      247983694.0
5             25-29            2545208491.0      196924943.0
6             30-34            2503870046.0      154941831.0
7             35-39            2414993437.0      122755538.0
8             40-44            2310941145.0       98369926.0
9             45-49            2179233684.0       78607475.0
10            50-54            2031594854.0       61767003.0
11            55-59            1846346004.0       47654243.0
12            60-64            1607991820.0       35805208.0
13            65-69            1363344713.0       25540584.0
14            70-74            1074644842.0       16577941.0
15            75-79     

In [ ]:

pop_est.loc[17,pop_est.columns[1:3]]=pop_est.loc[17:21,pop_est.columns[1:3]].sum()
pop_est.loc[17,pop_est.columns[0]]="85+"
pop_est=pop_est.drop(index=[18,19,20])


In [ ]:
print(pop_est)

   Age group(years) United States,1950-2021 Uganda,1950-2021
0               0-4            2665210640.0      519693594.0
1               5-9            2668027455.0      430624776.0
2             10-14            2679617305.0      364621694.0
3             15-19            2641486119.0      304857173.0
4             20-24            2577774727.0      247983694.0
5             25-29            2545208491.0      196924943.0
6             30-34            2503870046.0      154941831.0
7             35-39            2414993437.0      122755538.0
8             40-44            2310941145.0       98369926.0
9             45-49            2179233684.0       78607475.0
10            50-54            2031594854.0       61767003.0
11            55-59            1846346004.0       47654243.0
12            60-64            1607991820.0       35805208.0
13            65-69            1363344713.0       25540584.0
14            70-74            1074644842.0       16577941.0
15            75-79     

Population estimates based on 5-year age by groups and sex using variant Median (adjust to top="85+"

#===============DEADLINE TIME======================DEADLINE TIME=====

Extraction of Table 1 from the paper "AGE STANDARDIZATION OF RATES: A NEW WHO STANDARD, Authors: Omar B, Ahmad, Cynthia Boschi-Pinto, et.al. GPE Discussion Series of Paper: No 31 EIP/GPE/EBD World Health Organization 2001

In [ ]:
camelot_df = extract_table_from_pdf(Path,9)


age_st_cols=["Age group","Segi (“world”) standard","Scandinavian (“European”) standard", "WHO World Standard*"]
age_st=pd.DataFrame(camelot_df.loc[2:19,:].values,index=range(18),columns=age_st_cols)


print(age_st.head(18))
age_st.to_csv("age_st.csv")


   Age group Segi (“world”) standard Scandinavian (“European”) standard  \
0        0-4                   12.00                               8.00   
1        5-9                   10.00                               7.00   
2      10-14                    9.00                               7.00   
3      15-19                    9.00                               7.00   
4      20-24                    8.00                               7.00   
5      25-29                    8.00                               7.00   
6      30-34                    6.00                               7.00   
7      35-39                    6.00                               7.00   
8      40-44                    6.00                               7.00   
9      45-49                    6.00                               7.00   
10     50-54                    5.00                               7.00   
11     55-59                    4.00                               6.00   
12     60-64             

Extraction Table from the Instructions dcoument Data Analysis Exercise : Our Wolrd in Data Junior Data Scientist Application

In [ ]:

camelot_df = extract_table_from_pdf(Path1,3)

death_rate_cols=["Age group (years)" ,"Death rate, United States, 2019","Death rate, Uganda, 2019"]
death_rate=pd.DataFrame(camelot_df.loc[2:19,:].values,index=range(18),columns=death_rate_cols)

death_rate.to_csv("death_rate_.csv")

print(death_rate.head(18))


   Age group (years) Death rate, United States, 2019 Death rate, Uganda, 2019
0                0-4                            0.04                     0.40
1                5-9                            0.02                     0.17
2              10-14                            0.02                     0.07
3              15-19                            0.02                     0.23
4              20-24                            0.06                     0.38
5              25-29                            0.11                     0.40
6              30-34                            0.29                     0.75
7              35-39                            0.56                     1.11
8              40-44                            1.42                     2.04
9              45-49                            4.00                     5.51
10             50-54                           14.13                    13.26
11             55-59                           37.22            

# Calculation of crude death rate (CDR)

$\text{Crude Death Rate (CDR)} = \left( \frac{\text{Number of Deaths within a Year}}{\text{Mid-Year Population}} \right) \times 1000 $


The crude death rate is a fundamental statistic that helps in understanding the health status of a population, though it does not account for the age structure of the population, which can significantly affect the interpretation of this rate.


In [ ]:
scale=100000
crd_top=[]
for ii in range(death_rate.shape[1]-1):
    col=death_rate.columns[ii+1]
    col_=pop_est.columns[ii+1]
    crd_=death_rate[col].astype(float).dot(pop_est[col_].astype(float))/scale
    crd_/=pop_est[col_].sum()
    crd_*=scale
    crd_top.append(round(crd_,1))


crude_death=pd.DataFrame(crd_top).T
crude_death.columns=["Crude death rate, United States, 2019", "Crude death rate, Uganda, 2019"]

print(crude_death.head())


   Crude death rate, United States, 2019  Crude death rate, Uganda, 2019
0                                   42.8                             7.3


#Calculation of age-specific death rate (ASDR)


 $  \text{ASDR} = \frac{\sum (\text{Age-specific death rate} \times \text{Standard population in each age group})}{\text{Total standard population}} \times 100,000
 $

This formula will give you the age-standardized death rate, which is a weighted average of the age-specific death rates, where the weights are the proportions of each age group within the standard population. This rate can then be used to make fair comparisons across different populations or within the same population over time.

In [ ]:
scale=100000
crd_top=[]
for ii in range(death_rate.shape[1]-1):
    crd_top_=[]
    for jj in range(age_st.shape[1]-1):
        col=death_rate.columns[ii+1]
        col_=age_st.columns[jj+1]
        crd_=death_rate[col].astype(float).dot(age_st[col_].astype(float))/scale
        crd_/=age_st[col_].astype(float).sum()
        crd_*=scale
        crd_top_.append(round(crd_,1))
    crd_top.append(crd_top_)
#print(crd_top)

asdr=pd.DataFrame(crd_top)
asdr.columns=age_st.columns[1:]
asdr.index=["United States","Uganda"]

print(asdr.head())


               Segi (“world”) standard  Scandinavian (“European”) standard  \
United States                     23.0                                38.3   
Uganda                            23.3                                38.1   

               WHO World Standard*  
United States                 28.4  
Uganda                        28.7  
